In [58]:
!conda install html5lib

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - html5lib


The following packages will be UPDATED:

    certifi:         2020.4.5.1-py36h9f0ad1d_0 conda-forge --> 2020.4.5.1-py36_0
    openssl:         1.1.1g-h516909a_0         conda-forge --> 1.1.1g-h7b6447c_0

The following packages will be DOWNGRADED:

    ca-certificates: 2020.4.5.1-hecc5488_0     conda-forge --> 2020.1.1-0       

Preparing transaction: done
Verifying transaction: done
Executing transaction: done


Step 1: get data from the wiki url/ creat the target dataframe columns and give them a name. 
Step 2: The most import loop process to get data from the web - #inspect structure: div(class)->table(class)->  {thead()->tr()->th(class)} OR {tbody()->tr()->td(table elements)} / loop each elements into the table, which located at {tbody()->tr()->td(table elements)}
Step 3: Remove unwanted values / # remove Not assigned borough, both 'not assigned or = 0'

In [16]:
#import packages
import numpy as np 
import pandas as pd
import json 
import requests 
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import lxml

#STEP_ONE: get data from the wiki url
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
# get data from the wiki url
canada_list = BeautifulSoup(source, 'lxml') 
#creat the target dataframe columns and give them a name
toronto_list = pd.DataFrame(columns=['Postalcode','Borough', 'Neighborhood'])

#STEP_TWO: The most import loop process to get data from the web
#inspect structure: div(class)->table(class)->  {thead()->tr()->th(class)} OR {tbody()->tr()->td(table elements)}
content = canada_list.find('div', class_='mw-parser-output')
table = content.table.tbody #can't be writen as the same row from above row
postcode = 0
borough = 0
neighborhood = 0

#loop each elements into the table, which located at {tbody()->tr()->td(table elements)}
for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:  #the first element followed by <td> as postcode
            postcode = td.text.strip('\n').replace('/','')
            i = i + 1
        elif i == 1: #the second element followed by <td> as borough
            borough = td.text.strip('\n').replace('/','')
            i = i + 1
        elif i == 2: #the third element followed by <td> as neighborhood
            neighborhood = td.text.strip('\n').replace('/',',') #need to gather neighbors in one cell
    toronto_list = toronto_list.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

#STEP_THREE: Remove unwanted values
# remove Not assigned borough, both 'not assigned or = 0'
toronto_list = toronto_list[toronto_list.Borough!='Not assigned']
toronto_list = toronto_list[toronto_list.Borough!= 0]
toronto_list.reset_index(drop = True, inplace = True)

#
i = 0
for i in range(0,toronto_list.shape[0]):
    if toronto_list.iloc[i][2] == 'Not assigned':
        toronto_list.iloc[i][2] = toronto_list.iloc[i][1]
        i = i+1
        
#print the dataframe
df = toronto_list.groupby(['Postalcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [59]:
df.shape


(103, 6)

Use the csv to inport lat and lng / join two table together on Postalcode

In [ ]:

#use the csv to inport lat and lng
df_coord = pd.read_csv("http://cocl.us/Geospatial_data")
df_coord.rename(columns={'Postal Code':'Postalcode'}, inplace=True)

#join two table together on Postalcode
df = pd.merge(df, df_coord, how='inner', on = 'Postalcode')
#save the file incase you change the content by chance.
df.to_csv('toronto_df.csv')
df = pd.read_csv('toronto_df.csv')
df.head(12)

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         112 KB

The following NEW packages will be INSTALLED:

    branca:  0.4.0-py_0  conda-forge
    folium:  0.10.1-py_0 conda-forge
    vincent: 0.4.4-py_1  conda-forge


branca-0.4.0         | 26 KB     | ##################################### | 100% 
vincent-0.4.4        | 28 KB     | ##################################### | 100% 
folium-0.10.1        | 59 KB   

In [25]:
!conda install -c conda-forge geopy --yes

Solving environment: done

# All requested packages already installed.



In [60]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium



Examine the resulting dataframe

In [62]:
# Step_1 examine the resulting dataframe.
neighborhoods = df[['Borough','Neighborhood','Latitude','Longitude']]
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 10 boroughs and 103 neighborhoods.


Use geopy library to get the latitude and longitude values of Toronto

In [65]:

# Step_2 Use geopy library to get the latitude and longitude values of Toronto.
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer") #why it;s to_explorer?
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


Create map of Toronto using latitude and longitude values

In [66]:
# Step_3 create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'],neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Chose another object scaborough to analysis

In [67]:

#Step_4 Chose another object scaborough to analysis
scarborough_data = neighborhoods[neighborhoods['Borough'] == 'Scarborough'].reset_index(drop=True)
scarborough_data.head()


,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,Scarborough,Woburn,43.770992,-79.216917
4,Scarborough,Cedarbrae,43.773136,-79.239476


Get the lat and lng for Scarborough

In [68]:
#Step_5 get the lat and lng for Scarborough
address = 'Scarborough, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.773077, -79.257774.


Create map of Scarborough using latitude and longitude values

In [69]:
#Step_6 create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11.2)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Use Foursquare to explore arounds

In [70]:
#Step_7 Use  Foursquare to explor arrounds
CLIENT_ID = 'SZVB1O4YW3TIXXJUVXQZF2WI4WG0J3RY0LE3SOILEFY4QPH5'
CLIENT_SECRET = 'IZJOYIA33SCFAZ5TV4ZJMPNME3HIAWKLZJ4UAXB505ICNR0V'
VERSION = '20160604'
#get neighbor's name
scarborough_data.loc[0, 'Neighborhood']

'Malvern , Rouge'

Get neighbor's lat and lng

In [73]:
#Step_8 Get neighbor's lat and lng
neighborhood_latitude = scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern , Rouge are 43.8066863, -79.19435340000003.


Creat url to get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.

In [71]:
#Step_8 Creat url to get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=SZVB1O4YW3TIXXJUVXQZF2WI4WG0J3RY0LE3SOILEFY4QPH5&client_secret=IZJOYIA33SCFAZ5TV4ZJMPNME3HIAWKLZJ4UAXB505ICNR0V&v=20160604&ll=43.8066863,-79.19435340000003&radius=500&limit=100'

get results of url and define next step

In [74]:
#Step_9 get results of url and define next step
results = requests.get(url).json()
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056


In [37]:
#Step_10 Eventhough I only have one neighbor, I will still continue to analysis
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [75]:

from IPython.display import Image
Image(url = 'https://cdn1.imggmi.com/uploads/2019/10/9/9939d6c7a5933d5b90bc5678efb45069-full.png')

Get venues near scarborough

In [76]:
#Step_11 get venues near scarborough
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
scarborough_venues = getNearbyVenues(names=scarborough_data['Neighborhood'],
                                   latitudes=scarborough_data['Latitude'],
                                   longitudes=scarborough_data['Longitude']
                                  )

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge


Check the size of venues

In [78]:

#Step_12 Check the size of venues
print(scarborough_venues.shape)
scarborough_venues.head()

(90, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Guildwood , Morningside , West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
4,"Guildwood , Morningside , West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


How many venues each neighbor has

In [41]:
#Step_13 How many venues each neighbor has
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners , Tam O'Shanter , Sullivan",14,14,14,14,14,14
"Cliffside , Cliffcrest , Scarborough Village West",2,2,2,2,2,2
"Dorset Park , Wexford Heights , Scarborough Town Centre",5,5,5,5,5,5
"Golden Mile , Clairlea , Oakridge",9,9,9,9,9,9
"Guildwood , Morningside , West Hill",7,7,7,7,7,7
"Kennedy Park , Ionview , East Birchmount Park",6,6,6,6,6,6


In [79]:
#Step_13 how many unique catetories in those venues
print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

There are 53 uniques categories.


Analize each Neighbor

In [43]:
#Step_14 Analize each neighbor
# one hot encoding
scarborough_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
scarborough_onehot['Neighborhood'] = scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [scarborough_onehot.columns[-1]] + list(scarborough_onehot.columns[:-1])
scarborough_onehot = scarborough_onehot[fixed_columns]

scarborough_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Gym,Hakka Restaurant,History Museum,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Jewelry Store,Korean Restaurant,Latin American Restaurant,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Thai Restaurant,Train Station,Vietnamese Restaurant
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood , Morningside , West Hill",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Guildwood , Morningside , West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [80]:
scarborough_onehot.shape

(90, 54)

Group neighbor and confirm Size

In [81]:
#Step_15 Group neighbor and confirm Size
scarborough_grouped = scarborough_onehot.groupby('Neighborhood').mean().reset_index()
scarborough_grouped.shape

(16, 54)

print neighbor and it's top 5 venues

In [82]:
#Step_16 print neighbor and it's top 5 venues
num_top_venues = 5

for hood in scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = scarborough_grouped[scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0  Latin American Restaurant  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4        American Restaurant  0.00


----Birch Cliff , Cliffside West----
                   venue  freq
0           Skating Rink  0.25
1                   Café  0.25
2  General Entertainment  0.25
3        College Stadium  0.25
4    American Restaurant  0.00


----Cedarbrae----
                 venue  freq
0  Fried Chicken Joint  0.12
1     Hakka Restaurant  0.12
2          Gas Station  0.12
3               Bakery  0.12
4                 Bank  0.12


----Clarks Corners , Tam O'Shanter , Sullivan----
                 venue  freq
0             Pharmacy  0.14
1          Pizza Place  0.14
2  Fried Chicken Joint  0.07
3          Gas Station  0.07
4         Noodle House  0.07


----Cliffside , Cliffcrest , Scarborough Village West----
                 venue  freq
0  American Restaurant   0.5
1        

Put them into dataframe and get 10 venues for each neighborhood this time

In [83]:

#Step_17 put them into dataframe and get 10 venues for each neighborhood this time
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = scarborough_grouped['Neighborhood']

for ind in np.arange(scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,College Stadium,Gym,Grocery Store,General Entertainment,Gas Station
1,"Birch Cliff , Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Vietnamese Restaurant,Hakka Restaurant,Gym,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Department Store
3,"Clarks Corners , Tam O'Shanter , Sullivan",Pharmacy,Pizza Place,Noodle House,Italian Restaurant,Fast Food Restaurant,Coffee Shop,Fried Chicken Joint,Bank,Intersection,Gas Station
4,"Cliffside , Cliffcrest , Scarborough Village West",American Restaurant,Motel,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant


cluster the neighborhood into five clusters

In [84]:
#Step_18 cluster the neighborhood into five clusters
# set number of clusters
kclusters = 5

scarborough_grouped_clustering = scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int32)

New data for the following graph

In [53]:
#Step_19 new data for the following graph
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

scarborough_merged = scarborough_data

# merge scarborough_grouped with scarborough_data to add latitude/longitude for each neighborhood
scarborough_merged = scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

scarborough_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern , Rouge",43.806686,-79.194353,2.0,Fast Food Restaurant,Vietnamese Restaurant,Ice Cream Shop,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store
1,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,0.0,History Museum,Bar,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
2,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1.0,Intersection,Bank,Breakfast Spot,Rental Car Location,Electronics Store,Medical Center,Mexican Restaurant,Vietnamese Restaurant,Department Store,Grocery Store
3,Scarborough,Woburn,43.770992,-79.216917,4.0,Coffee Shop,Korean Restaurant,College Stadium,Hakka Restaurant,Gym,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant
4,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Bakery,Bank,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Vietnamese Restaurant,Department Store


In [55]:
scarborough_merged.dtypes

Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

In [85]:
#This role is for when you need to use the cluster for data, but color=rainbow[cluster-1] can't take float, then you have to convert the Cluster Labels into int.
scarborough_merged= scarborough_merged.fillna(0)
scarborough_merged[['Cluster Labels']] = scarborough_merged[['Cluster Labels']].astype("int")

View the clustered data

In [57]:

#Step_20 view the clustered data
# create map
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(scarborough_merged['Latitude'], scarborough_merged['Longitude'], scarborough_merged['Neighborhood'], scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters